In [ ]:
from huggingface_hub import login
import os
hf_token = hf_token = "YOUR_HF_TOKEN"
login(token=hf_token)

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


In [ ]:
modelName = "google/gemma-2-2b-it"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(modelName)

# 모델 로드 (양자화 설정 제거)
model = AutoModelForCausalLM.from_pretrained(
    modelName,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32  # dtype 명시적으로 설정
)

# 모델과 토크나이저가 올바르게 로드되었는지 확인
print(model)
print(tokenizer)

In [ ]:
# 1. CSV 파일 읽어오기
df = pd.read_csv('dataset.csv')

# 2. 챗봇의 답변이 비어있는 경우, '구분'이 같은 input에 대한 챗봇 답변을 채워넣기
def fill_missing_responses(row):
    if pd.isna(row['챗봇']):
        same_category = df[(df['구분'] == row['구분']) & (df['챗봇'].notna())]
        if not same_category.empty:
            return same_category.iloc[0]['챗봇']
    return row['챗봇']

df['챗봇'] = df.apply(fill_missing_responses, axis=1)

# 3. 데이터 토크나이즈하기
def tokenize_row(row):
    if pd.notna(row['유저']) and pd.notna(row['챗봇']):
        inputs = tokenizer(row['유저'], padding='max_length', truncation=True, max_length=128, return_tensors="np")
        labels = tokenizer(row['챗봇'], padding='max_length', truncation=True, max_length=128, return_tensors="np")
        print(f"Input: {row['유저']} -> Tokenized: {inputs['input_ids'][0].tolist()}")
        print(f"Label: {row['챗봇']} -> Tokenized: {labels['input_ids'][0].tolist()}")
        return pd.Series([inputs['input_ids'][0].tolist(), labels['input_ids'][0].tolist()])
    return pd.Series([None, None])

df[['input_ids', 'labels']] = df.apply(tokenize_row, axis=1)

# 4. tokenized_dataset 생성
tokenized_dataset = df[['input_ids', 'labels']].dropna().reset_index(drop=True)

In [ ]:
# Dataset 객체로 변환
dataset = Dataset.from_pandas(tokenized_dataset)

# 데이터셋 샘플링 (선택 사항: 데이터셋 크기를 줄여 메모리 사용량 감소)
dataset = dataset.shuffle(seed=42).select(range(1000))  # 예: 1000개의 샘플만 사용

# 훈련 데이터셋과 평가 데이터셋 분리
train_size = int(0.8 * len(dataset))
train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, len(dataset)))

In [ ]:
# LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

# 모델을 LoRA로 준비
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
print(model)

# 모델을 올바른 장치로 이동
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

In [ ]:
# 평가 메트릭 계산 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    loss = np.mean((predictions - labels) ** 2)  # 예시로 MSE 사용
    return {"eval_loss": loss}

In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",  # evaluation_strategy를 steps로 변경
    save_strategy="steps",  # save_strategy를 steps로 변경
    eval_steps=500,  # 500 스텝마다 평가
    save_steps=500,  # 500 스텝마다 체크포인트 저장
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # 배치 크기를 더 줄임
    num_train_epochs=3,
    weight_decay=0.01,
    remove_unused_columns=False,  # 이 부분을 추가하여 오류 해결
    bf16=torch.backends.mps.is_available(),  # Mixed Precision Training 활성화 (MPS에서만)
    save_total_limit=3,  # 최대 저장할 체크포인트 수
    load_best_model_at_end=True,  # 훈련 종료 시 가장 좋은 모델 로드
    metric_for_best_model="eval_loss",  # 가장 좋은 모델을 선택할 메트릭
    greater_is_better=False,  # 낮은 eval_loss가 더 좋음
    gradient_accumulation_steps=8,  # Gradient Accumulation 사용
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=eval_dataset,  # 평가 데이터셋 추가
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # 평가 메트릭 추가
)
# 모델 훈련 시작
checkpoint_path = "./results/"
if os.path.exists(checkpoint_path):
    trainer.train(resume_from_checkpoint=checkpoint_path)  # checkpoint-1000에서 재개
else:
    trainer.train()  # 처음부터 훈련 시작

In [ ]:
tmodel_path = "./results/final_model"  # 학습된 모델이 저장된 경로
tmodel = AutoModelForCausalLM.from_pretrained(tmodel_path)
tmodel.eval()

# 모델을 올바른 장치로 이동
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
tmodel.to(device)

In [ ]:
def generate_response_unfined(prompt, max_length=256):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=False,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(prompt):].strip()
    response = response.replace("**", "").strip()
    return response

def generate_response_fined(prompt, max_length=256):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = tmodel.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=False,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(prompt):].strip()
    response = response.replace("**", "").strip()
    return response

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response_unfined = generate_response_unfined(user_input, max_length=512)  # max_length를 늘려서 응답이 짧게 끊기지 않도록 함
    response_fined = generate_response_fined(user_input, max_length=512)  # max_length를 늘려서 응답이 짧게 끊기지 않도록 함
    print(f"Unfined Bot: {response_unfined}")
    print(f"Fined Bot: {response_fined}")